In [ ]:
## imporrt libarary 
import os
import pandas as pd
import numpy as np

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform
from support_functions import list_dtypes

## sklearn module
from xgboost.sklearn import XGBRegressor
import pickle

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "./sa-development.json"

### Configuration


In [ ]:
project_id = 'dti-ds'
dataset_id = 'putri_dataset_028'
table_id = 'test_data_bulk'
region = 'us-central1'
bucket_name = 'putri_gcs_028'
blob_name = 'data/test_data_bulk.csv'

In [ ]:
import os

model_name = "putri_Model_UsedCars_Saudi_Arabia.sav"
local_model_dir = "model"
local_model_path = os.path.join(local_model_dir, model_name)

# Ensure the local directory exists
os.makedirs(local_model_dir, exist_ok=True)

try:
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name)
    blob_model = bucket.blob(f"model/{model_name}")
    blob_model.download_to_filename('putri_Model_UsedCars_Saudi_Arabia.sav')

    print("Retrieve model succeeded")
except Exception as e:
    raise TypeError(f"An error occurred: {e}")

In [ ]:
import os

def find_file(filename, search_path):
    result = []
    for root, dirs, files in os.walk(search_path):
        if filename in files:
            result.append(os.path.join(root, filename))
    return result

model_name = "putri_Model_UsedCars_Saudi_Arabia.sav"
search_path = "."  # Current directory or specify the root directory to start the search

found_files = find_file(model_name, search_path)

if found_files:
    for file_path in found_files:
        print("Found:", file_path)
else:
    print(f"{model_name} not found in {search_path}")

### Model Predictions


import pickle
import sklearn
import xgboost

from sklearn.compose import ColumnTransformer



model_name = "putri_Model_UsedCars_Saudi_Arabia.sav"
model_path = "./putri_Model_UsedCars_Saudi_Arabia.sav"

loaded_model = pickle.load(open(model_path, 'rb'))

# Sample data for prediction
predict_car = pd.DataFrame({
    'Type':['Optima'],
    'Region':['Abha'], 
    'Make':['Kia'], 
    'Gear_Type':['Automatic'], 
    'Origin':['Saudi'], 
    'Options':['Semi Full'], 
    'Year':[2015],
    'Engine_Size':[2.0], 
    'Mileage':[10000]
})

loaded_model.predict(predict_car)




predictions = loaded_model.predict(predict_car)

predictions

In [ ]:
# load data from BQ
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
test_df = query_job.result().to_dataframe()

test_df

bulk_predict_df = test_df

In [ ]:
y_pred = loaded_model.predict(bulk_predict_df)
y_pred_df = pd.DataFrame(y_pred, columns=['Price Prediction'])

result_df = pd.concat([test_df.reset_index(drop=True), y_pred_df.reset_index(drop=True)], axis=1)
result_df